# TFRecord
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- Tensorflow에서 제공하는 데이터셋을 저장방식.
    - 데이터 양이 많을 경우 이를 Binary로 Seralization(직렬화)하여 하나의 파일로 저장하고 있다가, 이를 다시 읽어 들여  처리하면 처리속도를 향상시킬 수 있다. Tensorflow에서 이를 위해서 데이터 셋을 Protocol Buffer 형태로 Serialization을 수행해서 저장할 수 있는 TFRecords 파일 포맷 형태를 지원한다. 
    - tf.train.Example 클래스를 이용해서 {“string” : tf.train.Feature} 의 딕셔너리 형태로 데이터들을 TFRecords 파일에 저장할 수 있다.
- tf.train.Example
    - 하나의 데이터를 TFRecord에 저장하기 위해 변환하는 클래스. 하나의 데이터를 tf.train.Example 의 객체로 변환해서 저장한다.
- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세가지 타입을 지원한다.
        - tf.train.BytesList – string, byte 타입을 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List – bool, enum, int32, uint32, int64, uint64 타입을 변환
- tf.tran.Example의 형태
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```
> - **직렬화(Serialization):** 메모리에 저장된 다양한 타입의 값을 디스크(네트워크)에 저장할 수 있는 상태로 변환하는 것.
> - **binary data:** 디스크에 저장되는 0, 1로 구성된 데이터

In [1]:
import tensorflow as tf
import numpy as np

In [17]:
def _bytes_feature(value):
    """
    bytes string, bytes(이진파일) 타입의 value를 받아서 BytesList로 변환하는 함수
    """
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList는 EagerTensor타입은 변환할 수 없기 때문에 ndarray로 변환해야 한다.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value])) #value를 ByteList로 변환후 Feature타입으로 생성.
    

def _float_feature(value):
    """
    float타입의 value를 받아서 FloatList 변환 하는 함수
    """
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """
    int, uint, bool 타입의 value를 받아서 Int64List로 변환하는 함수
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [19]:
v = 30
r = _int64_feature(v)
print(type(r))
r

<class 'tensorflow.core.example.feature_pb2.Feature'>


int64_list {
  value: 30
}

In [20]:
v = 5.73
_float_feature(v)

float_list {
  value: 5.730000019073486
}

In [22]:
v = b"abc" #bytes str (문자열: str객체, byte str: binary code(0/1))
_bytes_feature(v)

bytes_list {
  value: "abc"
}

In [29]:
v = "안녕"
v2 = v.encode('utf-8') # 문자열(utf-8)을 bytes encoding
_bytes_feature(v2)

bytes_list {
  value: "\354\225\210\353\205\225"
}

In [28]:
"test".split('.')

['test']

In [32]:
with open('capture_images/one-c6c5141b-16a0-11ec-98a7-e82a44a807ba.jpg', 'rb') as f:
    img = f.read()
type(img), type(v2), type(b'abc')  

(bytes, bytes, bytes)

In [34]:
r = _bytes_feature(img)
type(r)

tensorflow.core.example.feature_pb2.Feature

### Feature 직렬화
- .SerializeToString()
    - proto 메세지를 bytes(binary string)로 직렬화
    - Example을 tfrecord로 출력하기 전에 변환해야 한다.

In [35]:
f = _float_feature(0.234)
f

float_list {
  value: 0.23399999737739563
}

In [36]:
f.SerializeToString()

b'\x12\x06\n\x04\xb2\x9do>'

# TFRecord 저장 예제

### tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 3 가지 호환 유형 중 하나를 포함하는 tf.train.Feature 로 변환(인코딩)되어야 한다.
2. Feature이름 문자열에 1번에서 에서 생성 된 인코딩 된 기능 값으로 딕셔너리를 생성한다.
3. 2 단계에서 생성 된 맵은 Features 메시지 로 변환한다.


In [39]:
N_DATA = 1000 # 1000개의 데이터포인트를 생성.

# TFRecord에 저장할 가상의 DataSet을 생성.
# bool
feature0 = np.random.choice([False, True], N_DATA)
# int
feature1 = np.random.randint(0,5, N_DATA)
#float
feature2 = np.random.normal(size=N_DATA) #평균 0, 표준편차 1 정규분포
# str
s_list = [b'cat', b'dog', b'lion', b'tiger']
feature3 = np.random.choice(s_list, N_DATA)

feature0.shape, feature1.shape, feature2.shape, feature3.shape, 

((1000,), (1000,), (1000,), (1000,))

In [46]:
def _serialize_example(f0, f1, f2, f3):
    """
    1개의 tf.train.Example을 생성한 뒤에 Serialize해서(파일로 출력할 수있는 상태로 변환) 반환하는 함수.
    [매개변수]
        f0, f1, f2, f3: 속성값들
    """
    feature = {
        "feature0":_int64_feature(f0),
        "feature1":_int64_feature(f1),
        "feature2":_float_feature(f2),
        "feature3":_bytes_feature(f3)
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

### 출력 처리
- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.

In [49]:
import os
tfr_dir = 'sample_tfrecord'
if not os.path.isdir(tfr_dir):
    os.mkdir(tfr_dir)

In [50]:
tfr_file_path = os.path.join(tfr_dir, 'data.tfr') #파일경로 : sample_tfrecord\data.tfr #확장자: tfr, record, tfrecord
print(tfr_file_path)

sample_tfrecord\data.tfr


In [51]:
# TFRecordWriter 생성 - 출력 Strean 객체
tf_writer = tf.io.TFRecordWriter(tfr_file_path)

In [64]:
# 출력작업
for data in zip(feature0, feature1, feature2, feature3):
    print(data)
    break
    example_serialized = _serialize_example(bool(data[0]), data[1], data[2], data[3]  )  #bool -> 파이썬 bool 타입으로 변환.
    tf_writer.write(example_serialized)
    
tf_writer.close()

(False, 0, 1.3398088539850546, b'dog')


# TFRecord파일 읽기 및 역직렬화(Deserialize)
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.

In [56]:
# bytes -> Example
tf.train.Example.FromString(example_serialized) #역직렬화

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "feature1"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "feature2"
    value {
      float_list {
        value: 0.8525660037994385
      }
    }
  }
  feature {
    key: "feature3"
    value {
      bytes_list {
        value: "dog"
      }
    }
  }
}

In [57]:
def _parse_example(serialized_example):
    """
    직렬화된(tfrecord에 저장된) example을 받아서 feature들을 추출하여 반환하는 함수
    [매개변수]
         serialized_example: bytes - 값을 추출할 직렬화된 example
    """
    
    # Example에서 읽어온 Feature들을 저장할 dictionary
    # "저장할때지정한 Feature name" : FixedLenFeature(빈 Feature)
    features = {
        "feature0":tf.io.FixedLenFeature([], tf.int64), 
        "feature1":tf.io.FixedLenFeature([], tf.int64),
        "feature2":tf.io.FixedLenFeature([], tf.float32),
        "feature3":tf.io.FixedLenFeature([], tf.string)
    }
    
    # 직렬화된 Example을 tf.train.Example객체로 변환해주는 함수 
    parsed_example = tf.io.parse_single_example(serialized_example, features)
    
    # Example에서 Feature를 추출한 다음에 파이썬 값으로 변환(형변환-tf.cast())
    f0 = tf.cast(parsed_example['feature0'], tf.bool)
    f1 = tf.cast(parsed_example['feature1'], tf.int64)
    f2 = tf.cast(parsed_example['feature2'], tf.float32)
    f3 = tf.cast(parsed_example['feature3'], tf.string)
    
    return f0, f1, f2, f3

In [58]:
# tfrecord 파일에서 읽기

In [65]:
dataset = tf.data.TFRecordDataset(tfr_file_path).map(_parse_example)

In [69]:
for d in dataset.take(3):
    print(d[0], d[1], d[2], d[3])

tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(0, shape=(), dtype=int64) tf.Tensor(1.3398088, shape=(), dtype=float32) tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(True, shape=(), dtype=bool) tf.Tensor(3, shape=(), dtype=int64) tf.Tensor(-1.2123506, shape=(), dtype=float32) tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(4, shape=(), dtype=int64) tf.Tensor(-0.42264977, shape=(), dtype=float32) tf.Tensor(b'tiger', shape=(), dtype=string)
